# Collaborative filtering using WALS algorithm | 1

## Create raw dataset

In [0]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [0]:
from google.cloud import bigquery
from google.colab import auth

In [0]:
auth.authenticate_user()

In [0]:
sql = """
#standardSQL
SELECT 
  Dataset_B.user_id as visitorId,
  Dataset_A.content_id as contentId,
  COUNT(Dataset_B.user_id) as numberOfVisits

  FROM AGEA_ASL.Dataset_A LEFT JOIN AGEA_ASL.Dataset_B 
  ON Dataset_A.content_id = Dataset_B.content_id
  GROUP BY contentId,visitorId
"""
df = client.query(sql + 'LIMIT 1000').to_dataframe() # TODO: SACAR LIMIT
df.head()

,visitorId,contentId,numberOfVisits
0,3354744,iSxtIl20T,1
1,6173819,YseZiW82j,1
2,4721998,a6OfN4vlk,1
3,3828869,vF3slXjBT,1
4,4873900,ZWc35_MGK,1


In [0]:
stats = df.describe()

,numberOfVisits
count,1000.000000
mean,1.027000
std,0.174073
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


In [0]:
# the rating is the numberOfVisits scaled to be in the range 0-1
median = stats['numberOfVisits']['50%']
df['rating'] = 0.3 * df['numberOfVisits'] / median
df[df['rating'] > 1]['rating'] = 1

In [0]:
del df['numberOfVisits']

In [0]:
!rm -rf data
!mkdir data

In [0]:
df.to_csv('data/collab_raw.csv', index=False, header=False)

In [0]:
# upload to bucket just in case that the local csv be erased
!gsutil cp 'data/collab_raw.csv'  gs://{BUCKET}/collab_raw.csv 

Copying file://data/collab_raw.csv [Content-Type=text/csv]...
/ [1 files][ 21.5 KiB/ 21.5 KiB]                                                
Operation completed over 1 objects/21.5 KiB.                                     


In [0]:
!head data/collab_raw.csv

3354744,iSxtIl20T,0.3
6173819,YseZiW82j,0.3
4721998,a6OfN4vlk,0.3
3828869,vF3slXjBT,0.3
4873900,ZWc35_MGK,0.3
5897171,e9Sq0o8Mz,0.3
4207853,qTIuTHv3D,0.3
6280619,6gNoVtDlH,0.3
3858068,dGiHrgot6,0.3
6429652,StXtMQRp6,0.3
